In [ ]:
import optuna
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import roc_auc_score
import numpy as np

# Ваши данные (замените X и y на реальные данные)
X = np.random.rand(40000, 10)  # Пример: 40 тысяч наблюдений, 10 признаков
y = np.random.choice([0, 1], size=40000, p=[0.8, 0.2])  # Дисбаланс классов

# Разделение на тренировочный и тестовый наборы
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

def objective(trial):
    """Функция для оптимизации гиперпараметров CatBoostClassifier."""
    # Гиперпараметры для подбора
    params = {
        'iterations': trial.suggest_int('iterations', 500, 2000),
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 0.3),
        'depth': trial.suggest_int('depth', 4, 10),
        'l2_leaf_reg': trial.suggest_loguniform('l2_leaf_reg', 1e-3, 10),
        'border_count': trial.suggest_int('border_count', 32, 255),
        'bagging_temperature': trial.suggest_uniform('bagging_temperature', 0.0, 1.0),
        'random_strength': trial.suggest_uniform('random_strength', 0.0, 10.0),
        'scale_pos_weight': trial.suggest_uniform('scale_pos_weight', 1.0, 10.0),
        'verbose': 0,
        'loss_function': 'Logloss',
        'eval_metric': 'AUC',
        'task_type': 'CPU',
    }

    # Кросс-валидация
    skf = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)
    auc_scores = []

    for train_idx, test_idx in skf.split(X_train, y_train):
        X_t, X_v = X_train[train_idx], X_train[test_idx]
        y_t, y_v = y_train[train_idx], y_train[test_idx]

        model = CatBoostClassifier(**params)
        model.fit(X_t, y_t, eval_set=[(X_v, y_v)], early_stopping_rounds=50, verbose=0)

        preds = model.predict_proba(X_v)[:, 1]
        auc = roc_auc_score(y_v, preds)
        auc_scores.append(auc)

    return np.mean(auc_scores)

# Запуск оптимизации
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=50, n_jobs=-1)

# Результаты
print("Best parameters:", study.best_params)
print("Best AUC:", study.best_value)

# Обучение с лучшими параметрами на полном тренировочном наборе
best_params = study.best_params
best_model = CatBoostClassifier(**best_params)
best_model.fit(X_train, y_train, eval_set=[(X_val, y_val)], early_stopping_rounds=50, verbose=100)

# Оценка модели
final_preds = best_model.predict_proba(X_val)[:, 1]
final_auc = roc_auc_score(y_val, final_preds)
print("Final AUC on validation set:", final_auc)
